# [![](https://img.shields.io/badge/Musubi-Tuner-Blue)](https://github.com/kohya-ss/sd-scripts/tree/sd3?tab=readme-ov-file#flux1-fine-tuning) [![](https://img.shields.io/badge/Video-Hướng%20dẫn-ff0000)](https://youtu.be/798qchqWty4) [![](https://img.shields.io/badge/Design-stablediffusion.vn-0075ff)](https://stablediffusion.vn) [![](https://img.shields.io/badge/Ver-1.0-0075ff)](https://trainlora.vn) [![](https://img.shields.io/badge/All%20Tools-trainlora.vn-0075ff)](https://trainlora.vn) [![](https://img.shields.io/badge/SDVN-Library-green)](https://bit.ly/sdvn-lib) [![](https://img.shields.io/badge/Khoá%20học-All%20in%20one-red)](https://hungdiffusion.com/) [![](https://img.shields.io/badge/Group-Support-0075ff)](https://www.facebook.com/groups/stablediffusion.vn) [![](https://img.shields.io/discord/813085864355037235?color=blue&label=Discord&logo=Discord)](https://discord.gg/5SEtApPeyG)
---

#⚙️ I. Cài đặt

In [ ]:
# @title ###Install
import os
from google.colab import drive

drive.mount('/content/drive')
root_dir = "/content/"
data_dir = "/content/drive/MyDrive/SD-Data"
repo_dir = "/content/SDVN-musubi-tuner-colab"

if not os.path.exists("/content/musubi-tuner"):
  %cd /content
  !git clone https://github.com/StableDiffusionVN/SDVN-musubi-tuner-colab
  !git clone https://github.com/kohya-ss/musubi-tuner
  %cd /content/musubi-tuner
  !sed -i 's/^bitsandbytes==0.45.0/bitsandbytes/' /content/musubi-tuner/requirements.txt
  !pip install -q -r /content/musubi-tuner/requirements.txt
  !pip install -q toml aria2 OpenAI google-genai flash_attn==2.7.3

%cd /content
%run {repo_dir}/TrainScript.ipynb

# 🎚️ II. Config Train

In [ ]:
#@title 📂 2. Kết nối - xử lý data

Image_train = True #@param {type:"boolean"}
ImageFolder = "/content/drive/MyDrive/SD-Data/TrainData"  # @param {type:'string'}
Video_train = False #@param {type:"boolean"}
VideoFolder = "/content/drive/MyDrive/SD-Data/TrainData"  # @param {type:'string'}
DataClean = False #@param {type:"boolean"}
No_gen_caption = False #@param {type:"boolean"}
# @markdown 🟡 `OCR Prompt`

# @markdown 💡 `Florence yêu cầu khởi động lại phiên (Ctr + M + .) và chạy lại từ đầu`

# @markdown 💡 `Tạo caption tự động cho video chỉ hỗ trợ dùng api qua gemini`
Caption_Image = 'None' # @param ['None', 'Florence', 'APIGemini | 2.0 Flash', 'APIGemini | 2.0 Flash Lite', 'APIOpenAI | GPT 4-o mini']
Caption_Video = 'None' # @param ['None','APIGemini | 2.0 Flash', 'APIGemini | 2.0 Flash Lite']
Caption_Length = "Short" # @param ["Short","Medium","Long"]
# @markdown 🟡 `API Caption`

# @markdown [![](https://img.shields.io/badge/Gemini-API-blue)](https://aistudio.google.com/app/apikey)
APIkey = "" # @param {type:'string'}
API_Prompt = "" # @param {type:'string'}
# @markdown 🟡 `Thêm caption tuỳ chọn`
Custom_Caption = "" # @param {type:'string'}
AddFolderName = False #@param {type:"boolean"}
Remove_Caption = False #@param {type:"boolean"}
Append = False #@param {type:"boolean"}

if No_gen_caption == True:
  Caption = 'None'
  Custom_Caption = ''
  AddFolderName = False

Cap_prompt = {
    'Short':['<CAPTION>',0.85, 30],
    'Medium':['<DETAILED_CAPTION>',0.5, 60],
    'Long':['<MORE_DETAILED_CAPTION>',0.35, 100]
}
extension = ".txt"

if DataClean == True :
  %cd /content
  clean_directory(ImageFolder)
if Caption_Image != "None" and Image_train:
  caption_dir(ImageFolder,Cap_prompt[Caption_Length])

if Caption_Video != "None" and Video_train:
  caption_dir(VideoFolder,Cap_prompt[Caption_Length], True)

list_train_dir = []
if Image_train:
  list_train_dir.append(ImageFolder)
if Video_train:
  list_train_dir.append(VideoFolder)
if AddFolderName:
  for dir in list_train_dir:
    add_forder_name(dir)
if Custom_Caption != "":
  for dir in list_train_dir:
    process_dir(dir, Custom_Caption, Append, Remove_Caption)


In [ ]:
#@title 🛠️ 3. Cấu hình train

%run {repo_dir}/Config.ipynb

model_train = "wan_i2v" # @param ["wan_t2v", "wan_i2v", "wan_t2v_13", "hun_t2v"]
output_dir = "/content/drive/MyDrive/SD-Data/Lora" # @param {type:'string'}
output_name = "lora_name" # @param {type:'string'}
#value
# @markdown 💡 `Tuỳ chỉnh theo tỉ lệ khung hình và lượng frame. Train hình tương đương frame = 1`
resolution = "720,480" # @param {type:'string'}
batch_size = 1 # @param {type:'number'}
image_dir = ImageFolder
video_dir = VideoFolder
num_repeats = 20 # @param {type:'number'}
max_train_epochs = 10 # @param {type:'number'}
save_every_n_epochs = 1 # @param {type:'number'}
save_last_n_epochs = 0 # @param {type:'number'}
save_every_n_steps = 0 # @param {type:'number'}
# @markdown 💡 `Các chế độ xử lý video`

# @markdown ▪️ `head - Lấy trực tiếp các frame từ đầu với lượng frame = target_frames`

# @markdown ▪️ `chunk - Chia video thành từng đoạn theo với lượng frame = target_frames`

# @markdown ▪️ `slide - Trích frame = target_frames và cách nhau frame_stride`

# @markdown ▪️ `uniform - Chia đều video thành frame_sample đoạn, mỗi đoạn trích suất frame = target_frames`

# @markdown ▪️ `full - Trích xuất toàn bộ video và giới hạn không quá max_frames`

# @markdown 💡 `target_frames - Lượng frame được cắt | VD:1,25,33 - 3 phương án với 1, 25, 33 frame`

frame_extraction = "head" # @param ["head","chunk","slide","uniform","full"]
target_frames = "33" # @param {type:'string'}
frame_stride = 1 # @param {type:'number'}
frame_sample = 1 # @param {type:'number'}
max_frames = 33 # @param {type:'number'}

# @markdown *️⃣ `Điều chỉnh Learning Rate`
learning_rate = 1e-4 # @param {type:'number'}
optimizer_type = "adamw8bit" # @param ["adamw", "adamw8bit", "adafactor"]
lr_scheduler = "constant" # @param [ "cosine", "constant", "polynomial"]
network_dim = 32 # @param {type:'number'}
network_alpha = 32 # @param {type:'number'}
# @markdown *️⃣ `Bật/Tắt các tính năng mở rộng, bao gồm (info, sampler, sao lưu, log)`
extra_option = True #@param {type:"boolean"}
author = '' # @param {"type":"string"}
description = '' # @param {"type":"string"}
sample_prompt = "" #@param {type:'string'}
sample_size = "512,512" #@param {type:'string'}
sample_every_n_steps = 200 # @param {type:'number'}
# @markdown 🟡 `Train model i2v (Image to video) sẽ cần một ảnh frame đầu để chạy được sample`
sample_image_i2v_path = "" #@param {type:'string'}
wandb_api_key = '' #@param {type:'string'}
save_state = False #@param {type:"boolean"}
save_state_on_train_end = False #@param {type:"boolean"}
resume = "" #@param {type:'string'}

#default

resolution = [int(x) for x in resolution.split(",")]
target_frames = [int(x) for x in target_frames.split(",")]
dataset = os.path.join(root_dir,"dataset.toml")

data_config = data_config(resolution, extension, batch_size)
dataset_file(Image_train, Video_train, image_dir, data_config, video_dir, model_train, num_repeats, frame_extraction, target_frames, frame_stride, frame_sample, max_frames)
config = config(model_train, optimizer_type, learning_rate, lr_scheduler, network_dim, network_alpha, max_train_epochs, save_every_n_epochs, save_last_n_epochs, save_every_n_steps, output_dir, output_name)
extra = extra(model_train, sample_every_n_steps, wandb_api_key, output_name, save_state, save_state_on_train_end, resume, author, description)
folder_train = ImageFolder if Image_train else VideoFolder
prompt(model_train, sample_prompt, folder_train, sample_image_i2v_path)
cache_latents_config = cache_latents_config(model_train)
cache_text_encoder_config = cache_text_encoder_config(model_train)

arg = f'accelerate launch {dic2arg(op)} {dic2arg(config)} {dic2arg(extra) if extra_option else ""}'
print(arg)
arg_latents_config = dic2arg(cache_latents_config)
print(arg_latents_config)
arg_text_encoder_config = dic2arg(cache_text_encoder_config)
print(arg_text_encoder_config)
print("Done")

# ⏳ III. Training

In [ ]:
#@title ⏳ 3. Training
%cd /content/musubi-tuner
run_cache = True #@param {type:"boolean"}
if run_cache:
  !{arg_latents_config}
  !{arg_text_encoder_config}
!{arg}

AutoDisconect = True # @param {"type":"boolean"}
if AutoDisconect:
  from time import sleep
  sleep(3*60)
  from google.colab import runtime
  runtime.unassign()